## Image Encryption

>This code is an based on the AES encryption algorithm in CBC mode.The program creates an encrypted and a decrypted file of the original image file. The process includes reading the file, converting it to base64, encrypting it, saving the encrypted file, decrypting it, and finally saving the decrypted file.

In [ ]:
#installing the pycryptodome package which includes the Random and AES modules
pip install pycryptodome

In [1]:
#importing required modules and library functions
import string
import base64 
from Crypto import Random
from Crypto.Cipher import AES
import hashlib

#constants
#block_size: defines the block sixe for AES encryption, which is 16bytes.
block_size = 16
pad_len=0

#SHA-256 hash of the provided key, ensuring the key is 32 bytes long, which is required for AES-256 encryption.
def sha256(key):
	sha = hashlib.sha256()
	sha.update(key.encode('utf-8'))
	return sha.digest()

#Padding: Adds padding to the plaintext to ensure its length is a multiple of the block size. PKCS7 padding is used here.
def pad(plain,block):
	pad_len = len(plain) % block
	return plain+((block-pad_len)*chr(block-pad_len)).encode('ascii')

#Unpadding: Removes the padding from the plaintext after decryption.
def unpad(plain,block):
	print(type(plain))
	return plain[0:-(block_size-pad_len)]

#Encryption: Pads the plaintext, generates a random IV, encrypts the plaintext using AES in CBC mode, and concatenates the IV with the encrypted text. 
#The result is base64 encoded for safe storage or transmission.
def encrypt(plain,key):
	plain = pad(plain,block_size)
	iv = Random.new().read(block_size)
	cipher = AES.new(key,AES.MODE_CBC,iv)
	final_cipher = cipher.encrypt(plain)
	return base64.b64encode(iv+final_cipher)

#Decryption: Base64 decodes the ciphertext, extracts the IV, decrypts the ciphertext, and removes the padding.
def decrypt(ciphertext,key):
	ciphertext = base64.b64decode(ciphertext)
	iv = ciphertext[:16]
	cipher = AES.new(key,AES.MODE_CBC,iv)
	plaintext = cipher.decrypt(ciphertext[16:])
	return unpad(plaintext,block_size) 


###Main

#prompts the user to enter the name of the file to be ncrypted and a key.
file = input('Enter name of the file: ')
key = input('Enter a key: ')

#hashes the key using SHA-256 to ensure it is the correct length for AES-256 encryption
key = sha256(key)

#opens file in binary read mode
fp = open(file,'rb')

#encodes the file contents to base64 to ensure the data is in a suitable format for encryption
base64_file = base64.b64encode(fp.read())

#Encrypts the base64-encoded file data using the provided key.
#Writes the encrypted data to a new file named encrypted.jpg.
enc = encrypt(base64_file,key)
fp1 = open("encrypted.jpg",'wb')
fp1.write(enc)

#Decrypts the encrypted data.
#Writes the decrypted data to a new file named decrypted.jpg.
dec = decrypt(enc,key)
fp2 = open('decrypted.jpg','wb')
fp2.write(base64.b64decode(dec))


fp.close()
fp1.close()
fp2.close()


Enter name of the file: bento.jpg
Enter a key: japanese food
<class 'bytes'>
